# Server Admin REST API

Using the **ArcGIS Server Administrator Directory** REST API to administer an ArcGIS Server and Services

## Imports

- ### **Requests**
  - for making HTTP requests
  - https://requests.readthedocs.io/en/latest/user/quickstart/

In [2]:
import json
from urllib.parse import urljoin

import requests

## Credentials

- Server Manager credentials are stored in a file outside of version control.  
- To get usernames and passwords, use the function below to get a dict with those values

In [3]:
def get_credentials():
    '''Get the username and password for the Server sites in a Dict with keys: 'user' and 'pswd'
    '''

    with open('params.json', 'r') as jsonfile:
        jsontxt = jsonfile.read()

        creds_json = json.loads(jsontxt)

    user = creds_json['login']['username']
    pswd = creds_json['login']['password']

    creds = {
        "user": user,
        "pswd": pswd
        }

    return creds

## Get the Server Administrator Root URL

- Each ArcGIS Server machine is has an Administrator Root URL for interacting with the ArcGIS REST API
- This is the starting point for all other API calls

https://developers.arcgis.com/rest/enterprise-administration/server/rootitems/

In [4]:
def get_root_url(portal, server, token=''):
    '''Returns the Server Administrator Root URL for a Server in a Portal
    '''

    # get Portal credentials and URL
    # stored in a dict in a json file stored outside version control

    with open('params.json', 'r') as jsonfile:
        jsontxt = jsonfile.read()

        creds = json.loads(jsontxt)

    # get the URL of the ArcGIS Server Administrator API root resource
    # https://developers.arcgis.com/rest/enterprise-administration/server/site/

    root_url = creds['roots'][portal][server]

    return root_url

## Generate a Token

- Any of the functions that operate on a password-protected REST API require a token for access
- Best practice in scripting is to generate a single token with the function below, then pass it as an argument to later functions.
- Each function has the ability to generate its own token, if none is passed.

https://developers.arcgis.com/rest/enterprise-administration/server/generatetoken/

In [5]:
def get_token(portal, server, exp_min=90):
    '''Returns a token that can be used by clients when working with the Server Admin API.
    '''

    # get Server credentials, stored outside version control

    creds = get_credentials()


    # build the URL to the 'Generate Token' endpoint
    # https://developers.arcgis.com/rest/enterprise-administration/server/generatetoken/

    token_subdir = "generateToken"
    root_url = get_root_url(portal, server)

    token_url = urljoin(root_url, token_subdir)

    # set the payload for the HTTP POST request
    # default expiration:  90 minutes

    payload = {
        "username": creds['user'],
        "password": creds['pswd'],
        "client": "requestip",
        "expiration": exp_min,
        "f": "json"
        }

    # send the POST requests using the Requests library
    r = requests.post(token_url, data=payload)

    # parse the token value out of the response JSON
    token = r.json()["token"]

    return token

### Check the validity of the token

Verify that a working token is available for the ArcGIS Server machine.  
- Why? To avoid generating a new token for every request.

Reasons why a token might not work:
- it has expired for the ArcGIS Server machine it was originally generated for
- it was generated for a different ArcGIS Server machine than the one it is attempting to be used with

This function sends a request to the root url
- if it comes back with an error, a new token is generated for the ArcGIS Server machine and returned
- if it works, the token is returned unchanged

In [6]:
def check_token(portal, server, token):

    # build URL for the ArcGIS Server REST endpoint
    # this endpoint requires a valid token, and returns a short JSON response, if successful

    subdir = "system"
    root_url = get_root_url(portal, server)

    test_url = urljoin(root_url, subdir)

    payload = {
        'token': token,
        'f': 'json'
        }

    r = requests.get(test_url, params=payload)
    # should return:  {'resources': ['directories', 'configstore', 'licenses']}`

    resp = r.json()

    if 'resources' in resp:

        # API returned the expected response, token is valid, return it
        return token

    if 'status' in resp:

        # API did not return the expected response
        # check if it has 'error' as its status

        # Token not found error
        # {'status': 'error', 'messages': ["Unauthorized access. Token not found. You can generate a token using the 'generateToken' operation."], 'code': 499}
        # Invalid token error
        # {'status': 'error', 'messages': ["Server machine returned an error. 'Invalid token.'"], 'code': 498}

        # any status is a sign of failed request, and there may be other token error codes

        if resp['code'] in (498, 499):

            # generate a new token and return it
            return get_token(portal, server)


## Machines

### Get the Machines in a Server site

- Each ArcGIS Server site will have one or more machines registered with it
- Most have a single machine, but "High Availability" Servers can have 2 or more in order to add more computing power

https://developers.arcgis.com/rest/enterprise-administration/server/machines/

In [7]:
def get_machines(portal, server, token=''):
    '''Returns a dict of the Machines registered with an ArcGIS Server site, at
    <root_url>/machines
    Normally, there is one machine per site.  A High-Availabilty site may have more than one.
    '''

    # build URL for the ArcGIS Server REST endpoint

    machines_subdir = "machines"
    root_url = get_root_url(portal, server)

    machines_url = urljoin(root_url, machines_subdir)

    # set the payload for the HTTP GET request
    # use the provided token argument, if present, or generate a new one

    payload = {
        'token': check_token(portal, server, token) if token else get_token(portal, server),
        'f': 'json'
        }

    r = requests.get(machines_url, params=payload)

    machines = r.json()

    return machines

### Get a Machine status

- Each Machine is either `Started` or `Stopped`
- This function returns the status of both the **Configured State** and the **Real-Time State**
  - The *configured* status represents the state of the resource as it was configured to be
  - The *real-time* status represents the actual state of a resource

In [8]:
def get_machine_status(portal, server, machine, token=''):
    '''Returns the Status of the specified Machine on the Server in Portal:
    {'configuredState': 'STARTED', 'realTimeState': 'STARTED'}
    '''

    # build URL to REST endpoint

    machine_status_subdir = f"machines/{machine}/status"
    root_url = get_root_url(portal, server)

    machine_status_url = urljoin(root_url, machine_status_subdir)

    payload = {
        'token': check_token(portal, server, token) if token else get_token(portal, server),
        'f': 'json'
        }

    r = requests.get(machine_status_url, params=payload)

    machine_status = r.json()

    return machine_status

# Services

## Get the Services in a Machine Root or Folder Path

### Get the Subfolders

In [9]:
def get_subfolders(portal, server, details=False, token=''):
    '''Returns a list of the subfolders in the root folder on the specified Machine
    on the specified Server in Portal
    Set details to 'True' to get a detailed dict of subfolders (folderName, description, etc.)
    Leave details as 'False' to get a simple list of folder name strings
    '''

    # build URL to REST endpoint

    services_subdir = f"services"

    root_url = get_root_url(portal, server)
    services_url = urljoin(root_url, services_subdir)

    payload = {
        'token': token if token else get_token(portal, server),
        'detail': 'true',
        'f': 'json'
        }

    r = requests.get(services_url, params=payload)

    if details == True:
        services = r.json()['foldersDetail']
    else:
        services = r.json()['folders']

    return services

### Get the Services in a folder

In [10]:
def get_services(portal, server, folder='', token=''):
    '''Returns a list of the Services (dicts) in the folder on the specified Machine on the Server in Portal
    Leave folder blank to get services in root
    '''

    # build URL to REST endpoint

    services_subdir = f"services{r'/' + folder}"   # any specified folder needs to begin with a forward slash '/'

    root_url = get_root_url(portal, server)
    services_url = urljoin(root_url, services_subdir)

    payload = {
        'token': token if token else get_token(portal, server),
        'detail': 'true',
        'f': 'json'
        }

    r = requests.get(services_url, params=payload)

    services = r.json()['services']

    return services

### Get a Service

In [11]:
def get_service(portal, server, folder, serviceName, serviceType, token=''):
    '''Returns a single Service (dict) as json text
    requires the Portal, Server, Folder, and Service names
    if service is in the root folder, use an empty string for folder
    '''

    # build URL to REST endpoint

    service_resource = f"services/{folder}/{serviceName}.{serviceType}"
    root_url = get_root_url(portal, server)

    service_url = urljoin(root_url, service_resource)

    payload = {
        'token': token if token else get_token(portal, server),
        'detail': 'true',
        'f': 'json'
        }

    r = requests.get(service_url, params=payload)

    service = r.json()

    return service

### Edit a Service

In [12]:
def edit_service(portal, server, folder, serviceName, serviceType, updates={}, token=''):
    '''edits the service to include the updates in the 'updates' dict'''

    # ex: updates = {'prop1':'new_value1', 'prop2':'new_value2', ...}

    # NOTE:
    # To edit a service, you need to submit the complete JSON representation of the service,
    # which includes the updates to the service properties.
    # Editing a service causes it to be restarted with updated properties.
    # use a POST request with a 'data' parameter (not 'params')

    if len(updates):  # at least one update exists

        # get the service as a dict
        service = get_service(portal, server, folder, serviceName, serviceType, token='')

        # update the service properties using the keys/values from the updates dict
        for prop in updates.keys():
            service[prop] = updates[prop]

        # service dict should now include the updated values

        # build URL to REST the /edit endpoint
        edit_path = f"services/{folder}/{serviceName}.{serviceType}/edit"
        root_url = get_root_url(portal, server)
        edit_url = urljoin(root_url, edit_path)

        # use json.dumps() to convert the dict to a JSON string
        payload = {
            'token': token if token else get_token(portal, server),
            'service': json.dumps(service),
            'f': 'pjson'
            }

        # send the edit request
        # for the POST request, uses 'data' instead of 'params' or you will get errors and errors and errors
        r = requests.post(edit_url, data=payload)

        return r.text

    else:
        return "No updates included"

## Make the Updates

In [ ]:
portal = "central"
server = "row"

token = get_token(portal, server, exp_min=90)

Update Services

- change any Service with a MaxInstancesPerNode value of 0 to a value of 1

In [17]:
def update_services(services, folder='', test=False):
    # runs through a list of services
    # this is done as a function in order to handle the list from the root
    # and the lists from the subfolders in one batch of code

    # set test param to True to see a list of services needing updates

    for service in services:

        if service['maxInstancesPerNode'] == 0:

            updates = {'maxInstancesPerNode': 1}  # new value
            serviceName = service['serviceName']
            serviceType = service['type']

            if test:
                # print the current service for testing / staging
                print(f"{portal:<12} {server:<12} {folder:<24}  {serviceName} ({serviceType})")
            else:
                # call the REST endpoint to update the service with new values
                # edit_service(portal, server, folder, serviceName, serviceType, updates, token)
                print("This was only a test")
                print(f"{portal:<12} {server:<12} {folder:<24}  {serviceName} ({serviceType}) {updates}")


## Fix (or Check) all servers on all portals

In [ ]:
test = True


with open('params.json', 'r') as jsonfile:
    jsontxt = jsonfile.read()

    j = json.loads(jsontxt)

deployments = j["roots"]

# print(servers)

for portal in deployments.keys():
    for server in deployments[portal].keys():
        print(f"{portal:<12}{server}")

        # get a token for this specific portal/server
        token = get_token(portal, server)

        # get the services in the root folder
        root_services = get_services(portal, server, '', token)

        # update the services in the root folder
        update_services(root_services)

        # get the names of the subfolders
        subfolders = get_subfolders(portal, server, False, token)

        # iterate through each folder
        for folder in subfolders:

            # get a list of the services
            # skip 'hosted' folder. those services don't show instance saturation metrics

            if not folder == "Hosted":
                services = get_services(portal, server, folder, token)

                # send that list to get updated
                # include True for testing. this will print the services, but leave them unchanged
                # use this to test for services that need to be updated

                update_services(services, folder, test)

        print("done\n")